In [1]:
%pylab
import tensorflow as tf
import pandas as pd

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
#load data and shuffle it
train_labeled = pd.read_hdf("train_labeled.h5", "train")
train_labeled=train_labeled.iloc[np.random.permutation(len(train_labeled))]

train_unlabeled = pd.read_hdf("train_unlabeled.h5", "train")
train_unlabeled=train_unlabeled.iloc[np.random.permutation(len(train_unlabeled))]

test = pd.read_hdf("test.h5", "test")

#slice labeled set into training and validation set
validation=train_labeled.iloc[:500]
train_labeled=train_labeled.iloc[500:]

In [3]:
# head is only there to only show the first five rows. without it you'd see the whole dataset
train_labeled.head()

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128
11625,7,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.777344,0.000000,0.0,0.0,0.992188,0.0,0.000000,0.0
3848,4,0.000000,0.0,0.000000,0.0,0.0,0.992188,0.414062,0.351562,0.0,...,0.0,0.0,0.261719,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
15079,5,0.609375,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.988281,0.0
7063,8,0.988281,0.0,0.984375,0.0,0.0,0.957031,0.250000,0.000000,0.0,...,0.0,0.0,0.164062,0.988281,0.0,0.0,0.000000,0.0,0.292969,0.0
12519,5,0.984375,0.0,0.472656,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.171875,0.859375,0.0,0.0,0.000000,0.0,0.000000,0.0


In [4]:
# just playing with pandas
train_labeled.describe()

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128
count,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,...,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.000000,8500.0,8500.000000,8500.000000
mean,4.490588,0.384602,0.000871,0.487811,0.036042,0.028556,0.491888,0.331702,0.134625,0.063714,...,0.014632,0.007489,0.501658,0.441359,0.003090,0.000759,0.079637,0.0,0.106024,0.000100
std,2.890768,0.433039,0.025540,0.431405,0.160712,0.142925,0.432394,0.421568,0.303752,0.214728,...,0.105642,0.074550,0.437681,0.430702,0.045984,0.021884,0.238977,0.0,0.273621,0.004974
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
50%,4.000000,0.082031,0.000000,0.500000,0.000000,0.000000,0.507812,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.544922,0.332031,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
75%,7.000000,0.945312,0.000000,0.984375,0.000000,0.000000,0.984375,0.851562,0.000000,0.000000,...,0.000000,0.000000,0.984375,0.976562,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
max,9.000000,0.996094,0.988281,0.996094,0.996094,0.996094,0.996094,0.996094,0.996094,0.996094,...,0.996094,0.996094,0.996094,0.996094,0.988281,0.996094,0.996094,0.0,0.996094,0.355469


In [5]:
# Format training data
DATA_x_train=train_labeled.loc[:,'x1':'x128'].values
tmp_DATA_y_=train_labeled['y'].values
DATA_y_=[]
for label in tmp_DATA_y_:
    DATA_y_.append(zeros(10))
    DATA_y_[-1][label]=1
DATA_y_

# Format unlabeled training data
DATA_x_train_u=train_unlabeled.loc[:,'x1':'x128'].values

# Format validation data
valDATA_x=validation.loc[:,'x1':'x128'].values
tmp_valDATA_y_=validation['y'].values
valDATA_y_=[]
for label in tmp_valDATA_y_:
    valDATA_y_.append(zeros(10))
    valDATA_y_[-1][label]=1

In [6]:
DATA_x_train_zusammen = DATA_x_train.tolist() + DATA_x_train_u.tolist()
DATA_x_train_zusammen = np.asarray(DATA_x_train_zusammen)
print(shape(DATA_x_train_zusammen))
"""
zero_cols=[]
DATA_x_train_zusammen=DATA_x_train
for i in range(128):
    # dimension level
    zero_counter=0
    for data_value in DATA_x_train_zusammen[:,i]:
        if data_value == 0.:
            zero_counter+=1
    if zero_counter!=0:
        zero_cols.append(zero_counter)
zero_cols"""

(29500, 128)


'\nzero_cols=[]\nDATA_x_train_zusammen=DATA_x_train\nfor i in range(128):\n    # dimension level\n    zero_counter=0\n    for data_value in DATA_x_train_zusammen[:,i]:\n        if data_value == 0.:\n            zero_counter+=1\n    if zero_counter!=0:\n        zero_cols.append(zero_counter)\nzero_cols'

In [ ]:
#Are there zero columns?
def is_zero_vector(x):
    count = 0
    for i in arange(len(x)):
        if x[i] == 0.:
            count += 1
    if count == len(x):
        return True
    return False

X_total = DATA_x_train.tolist() + DATA_x_train_u.tolist()
zero_i = list([])
for k in arange(len(X_total[0,:])):   
    if is_zero_vector(X_total[:,k]):
        zero_i.append(k)
zero_i

In [7]:
# Start tensorflow session
sess = tf.InteractiveSession()

In [8]:
# Abstract placeholders
x = tf.placeholder(tf.float32, shape=[None, 128])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [38]:
# Initializing model parameters, W weights and b biases
#W=tf.Variable(tf.zeros([128,10]))
#b=tf.Variable(tf.zeros([10]))

W=tf.Variable(tf.truncated_normal([128,10], stddev=0.1))
b=tf.Variable(tf.zeros([10]))

V=tf.Variable(tf.zeros([10,10]))
c=tf.Variable(tf.zeros([10]))

S=tf.Variable(tf.zeros([10,10]))
d=tf.Variable(tf.zeros([10]))

In [50]:
# Defining my network, including the loss function, cross_entropy
sess.run(tf.initialize_all_variables())
#y=tf.nn.softmax(tf.matmul(x,W)+b)
z=tf.nn.relu(tf.matmul(x,W)+b)
zz=tf.nn.relu(tf.matmul(z,S)+d)
y=tf.nn.softmax(tf.matmul(zz,V)+c)
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),reduction_indices=[1]))

In [54]:
# Gradient Descent with 0.5 step, run a given amount of times. NOTE: I removed the batching part that was in the tf tutorial.
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for i in range(1):
    #batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x:DATA_x_train,y_:DATA_y_})
    
# Evaluate based on the validation set that we separated in the beginning
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print(accuracy.eval(feed_dict={x:valDATA_x,y_:valDATA_y_}))

0.126


# K-means Scikit learn

In [113]:
initial_ids=np.zeros(10)
labels_tmp=train_labeled['y'].values
for i in range(len(labels_tmp)):
    for j in range(10):
        if labels_tmp[i]==j:
            initial_ids[j]=int(i)
initial_ids = [int(i) for i in initial_ids]
initial_ids
init=[]
for i in range(10):
    init.append(X[initial_ids[i]].tolist())
init=np.asarray(init)
init

array([[ 0.4375    ,  0.        ,  0.046875  , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.94140625, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.921875  ,  0.        ,  0.4921875 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.00390625, ...,  0.        ,
         0.        ,  0.        ]])

In [114]:
from sklearn.cluster import KMeans

X=DATA_x_train
Y=DATA_y_

estimator=KMeans(n_clusters=10,init=init,n_init=1,max_iter=1000000)
estimator.fit(X)
labels=estimator.labels_
tmp_labels=labels
PRED_labels=[]
for label in tmp_labels:
    PRED_labels.append(zeros(10))
    PRED_labels[-1][label]=1
no_correct=0
for j in range(len(PRED_labels)):
    label_vector=PRED_labels[j]
    correctness=1
    for i in range(len(label_vector)):
        if DATA_y_[j][i]!=label_vector[i]:
            correctness=0
    if correctness==1:
        no_correct+=1
accuracy=no_correct/shape(DATA_x_train)[0]
print(100*accuracy,"% accuracy")

52.247058823529414 % accuracy


In [127]:
labels=[]
for i in DATA_y_:
    for j in i:
        if j!=0:
            labels.append(j)
shape(labels)

(8500,)

In [29]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
data = scale(DATA_x_train)
pca = PCA(n_components=10).fit(data)
reduced_data = PCA(n_components=2).fit_transform(data)
shape(reduced_data)
for i in range(len(reduced_data)):
    plt.scatter(reduced_data[i][0],reduced_data[i][1])

/home/tamas/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/tamas/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/preprocessing/data.py:184: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [146]:
from time import time

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

np.random.seed(42)

#digits = load_digits()
data = scale(DATA_x_train)

n_samples, n_features = data.shape
n_digits = 10#len(np.unique(digits.target))

labels=[]
for i in DATA_y_:
    for j in i:
        if j!=0:
            labels.append(j)
            
#labels = DATA_y_#digits.target NOT GOOD TypE

sample_size = 300

print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))


print(79 * '_')
print('% 9s' % 'init'
      '    time  inertia    homo   compl  v-meas     ARI AMI  silhouette')

def accuracy(labels,labels_):
    
    no_correct=0
    for i in range(len(labels)):
        if labels[i]==labels_[i]:
            no_correct+=1
    acc=no_correct/len(labels)
    return acc


def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print("ACCURACY=",accuracy(labels,estimator.labels_))
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
              name="k-means++", data=data)

bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
              name="random", data=data)

# in this case the seeding of the centers is deterministic, hence we run the
# kmeans algorithm only once with n_init=1
pca = PCA(n_components=n_digits).fit(data)
bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
              name="PCA-based",
              data=data)
print(79 * '_')

###############################################################################
# Visualize the results on PCA-reduced data

reduced_data = PCA(n_components=2).fit_transform(data)
kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=10)
kmeans.fit(reduced_data)

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the digits dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

/home/tamas/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/tamas/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/preprocessing/data.py:184: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


n_digits: 10, 	 n_samples 8500, 	 n_features 128
_______________________________________________________________________________
init    time  inertia    homo   compl  v-meas     ARI AMI  silhouette
ACCURACY= 0.064
k-means++   2.43s    771306   1.000   0.000   0.000   0.000   0.000    0.006
ACCURACY= 0.22317647058823528
   random   1.72s    771685   1.000   0.000   0.000   0.000   0.000    0.038
ACCURACY= 0.06682352941176471
PCA-based   0.68s    779155   1.000   0.000   0.000   0.000   0.000    0.014
_______________________________________________________________________________


13.529411764705882 % accuracy


# K-means

In [ ]:
n_features = 128
n_clusters = 10

centroids, samples = create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed)

model = tf.initialize_all_variables()
with tf.Session() as session:
    sample_values = session.run(samples)
    centroid_values = session.run(centroids)

In [39]:
def create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed):
    np.random.seed(seed)
    slices = []
    centroids = []
    # Create samples for each cluster
    for i in range(n_clusters):
        samples = tf.random_normal((n_samples_per_cluster, n_features),
                               mean=0.0, stddev=5.0, dtype=tf.float32, seed=seed, name="cluster_{}".format(i))
        current_centroid = (np.random.random((1, n_features)) * embiggen_factor) - (embiggen_factor/2)
        centroids.append(current_centroid)
        samples += current_centroid
        slices.append(samples)
    # Create a big "samples" dataset
    samples = tf.concat(0, slices, name='samples')
    centroids = tf.concat(0, centroids, name='centroids')
    return centroids, samples

n_features = 2
n_clusters = 3
n_samples_per_cluster = 500
seed = 700
embiggen_factor = 70

np.random.seed(seed)
slices = []
centroids = []
# Create samples for each cluster
for i in range(n_clusters):
    samples = tf.random_normal((n_samples_per_cluster, n_features),
                           mean=0.0, stddev=5.0, dtype=tf.float32, seed=seed, name="cluster_{}".format(i))
    current_centroid = (np.random.random((1, n_features)) * embiggen_factor) - (embiggen_factor/2)
    centroids.append(current_centroid)
    samples += current_centroid
    slices.append(samples)
# Create a big "samples" dataset
samples = tf.concat(0, slices, name='samples')
centroids = tf.concat(0, centroids, name='centroids')
#print(shape(samples))
samples

<tf.Tensor 'samples_29:0' shape=(1500, 2) dtype=float32>

In [29]:
n_features = 2
n_clusters = 3
n_samples_per_cluster = 500
seed = 700
embiggen_factor = 70

np.random.seed(seed)

centroids, samples = create_samples(n_clusters, n_samples_per_cluster, n_features, embiggen_factor, seed)

model = tf.initialize_all_variables()
with tf.Session() as session:
    sample_values = session.run(samples)
    centroid_values = session.run(centroids)
    
sample_values=DATA_x_train
centroid_values=[DATA_x_train[np.random.choice(128,1)[0]] for i in range(10)]
print(shape(sample_values),shape(centroid_values))

(8500, 128) (10, 128)


In [42]:
def update_centroids(samples, nearest_indices, n_clusters):
    # Updates the centroid to be the mean of all samples associated with it.
    nearest_indices = tf.to_int32(nearest_indices)
    partitions = tf.dynamic_partition(samples, nearest_indices, n_clusters)
    new_centroids = tf.concat(0, [tf.expand_dims(tf.reduce_mean(partition, 0), 0) for partition in partitions])
    return new_centroids

def assign_to_nearest(samples, centroids):
    # Finds the nearest centroid for each sample

    # START from http://esciencegroup.com/2016/01/05/an-encounter-with-googles-tensorflow/
    expanded_vectors = tf.expand_dims(samples, 0)
    expanded_centroids = tf.expand_dims(centroids, 1)
    distances = tf.reduce_sum( tf.square(
               tf.sub(expanded_vectors, expanded_centroids)), 2)
    mins = tf.argmin(distances, 0)
    # END from http://esciencegroup.com/2016/01/05/an-encounter-with-googles-tensorflow/
    nearest_indices = mins
    return nearest_indices


x = tf.placeholder(tf.float32, shape=[None, 128])

sample_values=DATA_x_train
initial_centroids=[DATA_x_train[np.random.choice(128,1)[0]] for i in range(10)]

model = tf.initialize_all_variables()

with tf.Session() as session:
    for i in range(1000):
        updated_centroid_value = session.run(initial_centroids)
        update_centroids(sample_values,assign_to_nearest(sample_values,update_centroid_value),10)

TypeError: Fetch argument array([ 0.7890625 ,  0.        ,  0.59375   ,  0.        ,  0.        ,
        0.984375  ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.5546875 ,  0.        ,  0.59765625,  0.        ,  0.        ,
        0.984375  ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.91796875,  0.        ,  0.        ,  0.        ,
        0.765625  ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.09765625,  0.984375  ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.98828125,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.9296875 ,  0.984375  ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.38671875,  0.984375  ,  0.953125  ,  0.82421875,  0.        ,
        0.984375  ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.95703125,  0.        ,  0.98